In [1]:
def text_to_code_q10_2_binary(raw_ans):
    """
    Converts Q10_2 from 5-category responses to binary:
    - "Strongly disagree", "Somewhat disagree", "Neither agree nor disagree" → 0 (Not adopting)
    - "Somewhat agree", "Strongly agree" → 1 (Adopting)
    """
    adopting = {"Somewhat agree", "Strongly agree"}
    return 1 if raw_ans in adopting else 0

In [ ]:
TRANSFORMERS_q6a["Q10_2"] = text_to_code_q10_2_binary